In [1]:
# IMPORT
import numpy as np
import wandb
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import shutil
import os                              # Import the 'os' module for changing directories
os.chdir('/content/drive/MyDrive/FL')  # Change the directory

Mounted at /content/drive


In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR100
from torch.utils.data import Subset, DataLoader, random_split

from FederatedLearningProject.data.cifar100_loader import get_cifar100, create_iid_splits, create_non_iid_splits
import FederatedLearningProject.checkpoints.checkpointing as checkpointing
from FederatedLearningProject.training.FedMETA import aggregate_with_task_arithmetic, aggregate_masks_by_sparsity, aggregate_masks, distribution_function, train_server
from FederatedLearningProject.training.model_editing import plot_all_layers_mask_sparsity

from FederatedLearningProject.experiments import models
import copy

In [3]:
import importlib

# Importa i moduli del tuo progetto
from FederatedLearningProject.data import cifar100_loader
from FederatedLearningProject import checkpoints
from FederatedLearningProject.training import FedMETA, model_editing
from FederatedLearningProject import experiments

# Ricarica solo i moduli custom (NO torch)
importlib.reload(cifar100_loader)
importlib.reload(checkpoints.checkpointing)
importlib.reload(FedMETA)
importlib.reload(model_editing)
importlib.reload(experiments.models)


# Re-bind: importa di nuovo funzioni/classi/alias che usi nel codice
from FederatedLearningProject.data.cifar100_loader import (
    get_cifar100, create_iid_splits, create_non_iid_splits
)

import FederatedLearningProject.checkpoints.checkpointing as checkpointing

from FederatedLearningProject.training.FedMETA import (
    aggregate_with_task_arithmetic,
    aggregate_masks,
    distribution_function,
    train_server
)

from FederatedLearningProject.training.model_editing import (
    plot_all_layers_mask_sparsity
)

from FederatedLearningProject.experiments import models


In [4]:
wandb.login() # Ask for your APIw key for logging in to the wandb library.

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: niccolotozzi-principale (niccolotozzi-principale-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
device = "cuda"
model_name = "dino_vits16_J4"
project_name = "FederatedProjectPROVA_PARTE4"

# Generate a unique run name for each iteration
run_name = f"{model_name}_rounds_prova"
# INITIALIZE W&B for each new run
wandb.init(
    project=project_name,
    name=run_name,
    config={
        "model": model_name,
        "num_rounds": 100, # Use the current num_rounds_val
        "batch_size": 128, # Using test_loader's batch_size as a placeholder
    },
    reinit=True # Important: Allows re-initialization of wandb in a loop
)

# Copy your config
config = wandb.config

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


In [6]:
# print the content of the folder FederatedLearningProject.data.masks
print(os.listdir('FederatedLearningProject/masks'))

val_set = torch.load('FederatedLearningProject/masks/val_set.pth', weights_only=False)
train_set = torch.load('FederatedLearningProject/masks/train_set.pth', weights_only=False)
test_set = torch.load('FederatedLearningProject/masks/test_set.pth', weights_only=False)

['train_set.pth', 'val_set.pth', 'test_set.pth', 'client_masks_iid.pth', 'client_masks_non_iid_1.pth', 'client_masks_non_iid_5.pth', 'client_masks_non_iid_10.pth', 'client_masks_non_iid_50.pth', 'light_weight_masks_non_iid_1.pth', 'PORCODDIO.pth', 'light_weight_masks_iid.pth', 'light_weight_masks_non_iid_5.pth', 'light_weight_masks_non_iid_10.pth', 'light_weight_masks_non_iid_50.pth']


In [7]:
o_model = models.LinearFlexibleDino(num_layers_to_freeze=12)
local_masks = torch.load('FederatedLearningProject/masks/light_weight_masks_non_iid_1.pth')

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall16_pretrain.pth
100%|██████████| 82.7M/82.7M [00:01<00:00, 44.9MB/s]


In [8]:
model = copy.deepcopy(o_model)
model_checkpoint = torch.load("FederatedLearningProject/checkpoints/FL_NON_IID(1)/dino_vits_16_non_iid(1)_local_steps_4_checkpoint.pth")

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
model.load_state_dict(model_checkpoint['model_state_dict'])
#model.debug()

In [ ]:
model.unfreeze(12)
#model.debug()

In [ ]:
final_mask = aggregate_masks_by_sparsity(local_masks, sparsity_target = 0.9)
del local_masks

In [ ]:
final_mask_2 = aggregate_masks(local_masks, 0.5)

In [ ]:
partition_masks = distribution_function(final_mask, number_clients=100)

In [ ]:
client_dataset = create_non_iid_splits(train_set, num_clients=100, classes_per_client=1)    # each client will see only 1 class

In [ ]:
optimizer_config = {
    'lr': 0.01,
    'momentum': 0.9,
    'weight_decay': 0.0001
}

model.to_cuda()

checkpoint_path = 'FederatedLearningProject/checkpoints/'
val_loader = DataLoader(val_set, batch_size=128, shuffle=True)
criterion = nn.CrossEntropyLoss()

train_server(model, num_rounds=100, client_dataset=client_dataset, client_masks=partition_masks, optimizer_config=optimizer_config, device='cuda', frac=0.1, batch_size=128, val_loader=val_loader, checkpoint_path=checkpoint_path, criterion=criterion)